In [1]:
# Import libraries
## for profiling
from profiler.main import profile

# for titiler-xarray
from typing import Any
from titiler.xarray.factory import xarray_open_dataset, update_dataset
import morecantile
from rio_tiler.io import XarrayReader

# for titiler-pgstac
from titiler.pgstac.mosaic import PGSTACBackend
from typing import Any, Dict, List, Tuple
from rio_tiler.mosaic import mosaic_reader
from rio_tiler.models import ImageData
from geojson_pydantic import Polygon

# Only if wanting to see a picture
import io
from PIL import Image

In [2]:
search_dict: Dict[str, Any] = {
    'collections': ["CMIP6_ensemble_median_TAS"],
    'datetime': "1951-01-01T00:00:00Z",
    'bbox': [-180, -90, 180, 90]
}

xyz_tile = (0,0,0)

## Profile titiler-xarray

In [7]:
@profile(add_to_return=True, quiet=False, log_library='s3fs')
def tile(src_path: str, x: int, y: int, z: int, *, variable: str, time_slice: str = None, **kwargs: Any):

    with xarray_open_dataset(
        src_path,
        z=z,
        decode_times=False,
        **kwargs,
    ) as dataset:
        dataarray, _ = update_dataset(dataset, variable=variable, time_slice=time_slice)
        
        with XarrayReader(dataarray) as src_dst:
            return src_dst.tile(x, y, z)

In [8]:
image_and_assets, logs = tile(
    "s3://power-analysis-ready-datastore/power_901_monthly_meteorology_utc.zarr", 
    *xyz_tile,
    #time_slice = search_dict['datetime'],
    reference=False,
    variable="TS",
)
logs

Time elapsed for xarray.open_datset: 707.39603
Time elapsed for adjusting longitudes: 0.54288
Time elapsed for writing CRS: 10.36215
Time elapsed for time slice: 70.41192
in xarrayreader
Time elapsed for xy bounds: 0.06485
Time elapsed for clip_box: 16.72697


2023-05-31 14:59:41.843 | INFO     | profiler.main:wrapped_f:106 - {}


Time elapsed for reproject: 13694.37289
Time elapsed for ImageData: 0.51522
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/.zmetadata'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/lat/0'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/lon/0'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/time/0'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/time/1'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/time/10'}
CALL: get_object - () - {'Bucket': 'power-analysis-ready-datastore', 'Key': 'power_901_monthly_meteorology_utc.zarr/time/11'}
CALL: get_object - () - {'Bucket': 'power-ana

{}

In [ ]:
import fsspec
import xarray as xr
fs_read = fsspec.filesystem("s3", anon=True, skip_instance_cache=False)
ds=xr.open_dataset(
    "s3://power-analysis-ready-datastore/power_901_monthly_meteorology_utc.zarr", 
    engine='zarr', consolidated=True)
ds

In [ ]:
content = image_and_assets.render(
    img_format='PNG'
)

im = Image.open(io.BytesIO(content))
im

## Profile titiler-pgstac

In [ ]:
pool = tile_pgstac.connection_pool()

@profile(add_to_return=True, quiet=True, log_library = 'rasterio')
def tile(
    tile_x: int,
    tile_y: int,
    tile_z: int,
) -> Tuple[ImageData, List[str]]:
    
    backend = PGSTACBackend(pool=pool, input=tile_pgstac.mosaic_id(search_dict=search_dict, pool=pool))
    bbox = backend.tms.bounds(morecantile.Tile(tile_x, tile_y, tile_z))
    mosaic_assets = backend.get_assets(Polygon.from_bounds(*bbox))    

    def _reader(
        item: Dict[str, Any], x: int, y: int, z: int, **kwargs: Any
    ) -> ImageData:
        with backend.reader(item, tms=backend.tms, **backend.reader_options) as src_dst:
            return src_dst.tile(x, y, z, **kwargs)

    return mosaic_reader(mosaic_assets, _reader, tile_x, tile_y, tile_z, threads=10, **{"assets": ["data"]})  

In [ ]:
image_and_assets, logs = tile(*xyz_tile)
logs

In [ ]:
content = image_and_assets[0].render(
    img_format='PNG'
)

im = Image.open(io.BytesIO(content))
im